# packages and libraries

In [0]:
!pip install geopandas
import geopandas as gp

     |████████████████████████████████| 931kB 3.2MB/s 
     |████████████████████████████████| 14.7MB 288kB/s 
     |████████████████████████████████| 10.9MB 43.0MB/s 


In [0]:
!pip install censusdata
import censusdata as cd

     |████████████████████████████████| 23.2MB 110kB/s 
  Created wheel for censusdata: filename=CensusData-1.7-cp36-none-any.whl size=24706084 sha256=51a05c071613416ad4ef96470b0f029370ea34c3711d7f2841fb40c7605690b8
  Stored in directory: /root/.cache/pip/wheels/e8/9e/f9/8d0b054be9981c6f675630de9f32ce59620f8b515c13542a4c
Successfully built censusdata


In [0]:
# -- import library that adjusts USD for inflation using the Consumer Price Index, sourced from Department of Labor
# -- https://github.com/datadesk/cpi
!pip install cpi
import cpi

# -- download latest data
cpi.update()

     |████████████████████████████████| 25.5MB 1.5MB/s 


/usr/local/lib/python3.6/dist-packages/cpi/__init__.py:46: StaleDataWarning: CPI data is out of date. To accurately inflate to today's dollars, you must run `cpi.update()`.
  warnings.warn(StaleDataWarning())


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from zipfile import ZipFile
from io import BytesIO
from urllib.request import urlopen
from math import *
import requests
import re

# NOAA data

## shapefile

In [0]:
# -- get the zipfile from Census for the county boundaries shapefiles
shp_url = 'http://www2.census.gov/geo/tiger/GENZ2015/shp/cb_2015_us_county_500k.zip'

shp_req = requests.get(shp_url)
z_shp = ZipFile(BytesIO(shp_req.content))
z_shp.extractall(path='tmp/')

# -- get the filenames
shp_filenames = [y for y in sorted(z_shp.namelist()) for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)] 
print(shp_filenames)

['cb_2015_us_county_500k.dbf', 'cb_2015_us_county_500k.prj', 'cb_2015_us_county_500k.shp', 'cb_2015_us_county_500k.shx']


In [0]:
# -- save temporarily and read to merge with following dataframe
dbf, prj, shp, shx = [filename for filename in shp_filenames]
co_shp = gp.read_file('tmp/' + shp)

In [0]:
# -- change GEOID to int so that we can easily merge
co_shp["GEOID"] = co_shp["GEOID"].astype(int)

In [0]:
# -- dataframe with just county FIPS
co_fips = pd.DataFrame(co_shp["GEOID"].values, columns = ["FIPS"])
co_fips

,FIPS
0,1005
1,1023
2,1035
3,1051
4,1065
...,...
3228,45019
3229,45077
3230,46123
3231,47073


## NOAA damage

In [0]:
# -- Grab zone-county correlation file from NOAA and add columns
zcfile = "https://www.weather.gov/source/gis/Shapefiles/County/bp03mr20.dbx"
ztoc = pd.read_csv(zcfile, sep="|")

# -- add column names
ztoc.columns = ["state", "zone_num", "CWAID", "zone_name", "state_zone", "county", "c_fips", "timezone", "fe_area", "lat", "lon"]

In [0]:
# ztoc.to_csv(r'/content/drive/My Drive/MLPP20 Project Data/zone-county-NOAA.csv', index = False)

In [0]:
# -- combine zone fips with the corresponding state fips, so it'll match with noaa data
ztoc['actual_state_fips'] = ztoc['c_fips'].astype(str).str[-5:-3]
ztoc['zone_num'] = ztoc['state_zone'].str[-3:]
ztoc['zone_fips'] = (ztoc['actual_state_fips']+ztoc['zone_num']).astype(int)

# -- remove unnecessary columns
ztoc.drop(['CWAID', 'timezone', 'fe_area'], axis=1, inplace=True)

In [0]:
urlpath =urlopen('ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/')
string = urlpath.read().decode('utf-8')

pattern = re.compile(r'\bStormEvents_details-ftp_v1.0_d\S*?.gz') 
filelist = pattern.findall(string)

# -- get filename starting from 1996
csv_end = filelist[46:70]
mybegin = 'ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/'
csv = [mybegin + x for x in csv_end]
# csv = ["ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d1996_c20170717.csv.gz","ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d1997_c20190920.csv.gz","ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d1998_c20170717.csv.gz", "ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d1999_c20190920.csv.gz", "ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2000_c20190920.csv.gz", "ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2001_c20190920.csv.gz", "ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2002_c20190920.csv.gz", "ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2003_c20190920.csv.gz", "ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2004_c20190920.csv.gz", "ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2005_c20190920.csv.gz", "ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2006_c20190920.csv.gz", "ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2007_c20170717.csv.gz", "ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2008_c20180718.csv.gz", "ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2009_c20180718.csv.gz", "ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2010_c20191116.csv.gz", "ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2011_c20180718.csv.gz", "ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2012_c20200317.csv.gz", "ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2013_c20170519.csv.gz", "ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2014_c20191116.csv.gz", "ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2015_c20191116.csv.gz", "ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2016_c20190817.csv.gz", "ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2017_c20200121.csv.gz", "ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2018_c20200317.csv.gz", "ftp://ftp.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d2019_c20200416.csv.gz" ]

# -- list of years, until 2019
year = list(range(1996,2020))

In [0]:
# -- accumulate sum of missing damages
missing_dmg = 0

# -- loops through csv list for every year's file in noaa's website 
for i in range(len(csv)):
  # damage year column header as a string
  dmgyr = "dmg"+str(year[i])

  # read the noaa csv file for the given year
  noaa_all = pd.read_csv(csv[i], skiprows=0)

  # pull data from storms events that influence flood damage
  noaa_all = noaa_all[(noaa_all["EVENT_TYPE"] == "Coastal Flood") | 
                    (noaa_all["EVENT_TYPE"] ==  "Flash Flood") | 
                    (noaa_all["EVENT_TYPE"] == "Flood") | 
                    (noaa_all["EVENT_TYPE"] == "Heavy Rain") | 
                    (noaa_all["EVENT_TYPE"] == "High Wind") | 
                    (noaa_all["EVENT_TYPE"] ==  "Hurricane") | 
                    (noaa_all["EVENT_TYPE"] == "Lakeshore Flood") | 
                    (noaa_all["EVENT_TYPE"] == "Marine High Wind") | 
                    (noaa_all["EVENT_TYPE"] ==  "Marine Strong Wind") | 
                    (noaa_all["EVENT_TYPE"] == "Marine Thunderstorm Wind") | 
                    (noaa_all["EVENT_TYPE"] == "Seiche") | 
                    (noaa_all["EVENT_TYPE"] == "Storm Surge/Tide") | 
                    (noaa_all["EVENT_TYPE"] == "Thunderstorm Wind") | 
                    (noaa_all["EVENT_TYPE"] == "Tropical Depression") | 
                    (noaa_all["EVENT_TYPE"] == "Tropical Storm")]

  # extract state FIPS, county name, county FIPS, and property damage values
  noaa_all = noaa_all[['STATE_FIPS','CZ_TYPE','CZ_FIPS','CZ_NAME','DAMAGE_PROPERTY']].copy()

  # -- removes Puerto Rico, Virgin Islands, American Samoa, Guam, Northern Mariana Islands, and all bodies of water (ATLANTIC NORTH, ATLANTIC SOUTH,
  # -- E PACIFIC, GULF OF MEXICO, HAWAII WATERS, LAKE HURON, LAKE ONTARIO, LAKE MICHIGAN, LAKE ERIE, LAKE SUPERIOR, LAKE ST CLAIR, )
  noaa_all= noaa_all[noaa_all["STATE_FIPS"].astype(int) < 57]
  #print(noaa_all['STATE_FIPS'].astype(int).max())

  # convert DAMAGE_TYPE values from string to int
  noaa_all[dmgyr] = (noaa_all['DAMAGE_PROPERTY'].replace(r'[KMB]+$', ' ', regex=True).replace(' ',np.nan).astype(float) * noaa_all['DAMAGE_PROPERTY'].str.extract(r'[\d\.]+([KM]+)',expand=False).fillna(1).replace(['K','M','B'],[10**3,10**6,10**9]).astype(int))
  # -- Drop storms with na in damage column
  noaa_all = noaa_all.dropna(subset=[dmgyr])

  # construct a state+county ID, so all counties have unique identifier (combines state + county/zone code)
  state = noaa_all["STATE_FIPS"].astype(int).astype(str)
  county = noaa_all["CZ_FIPS"].astype(int).astype(str)
  county = "00"+ county 
  county = county.str[-3:]
  geoid = (state + county).astype(int)
  noaa_all["STATE_CO_FIPS"] = geoid

  # -- dataframe with just zone storm events
  noaa_z = noaa_all[noaa_all['CZ_TYPE'] == 'Z'][['STATE_CO_FIPS',dmgyr,'CZ_NAME']]

#-------------------------------------------------------------------------------------
###NOAA zone fips to zone->county county fips

  # group by zone FIPS and sum total damages per county for this year
  noaa_zg = noaa_z.groupby("STATE_CO_FIPS").sum().reset_index()

  # -- co_fips["FIPS"] not in "STATE_CO_FIPS" ~~~~~~these are the noaa zone FIPS that don't match with the zone->county zone fips
  noaa_zg[~noaa_zg["STATE_CO_FIPS"].isin(ztoc['zone_fips'])]["STATE_CO_FIPS"].count()

  # -- merge noaa data with zone-to-county dataframe on state+zone fips
  noaa_zm = noaa_zg.merge(ztoc,left_on="STATE_CO_FIPS",right_on='zone_fips',how='left') 

  #This calculates the amount of damages that get dropped because the zone fips don't match
  noaa_zm_null = noaa_zm[noaa_zm['state'].isnull()]
  noaa_zm_null_sum = noaa_zm_null[dmgyr].sum()
  missing_dmg += noaa_zm_null_sum

  # -- only select county fips and damage columns
  noaa_zm = noaa_zm[['c_fips',dmgyr]]
  # -- rename county fips column
  noaa_zm.columns = ['CO_FIPS',dmgyr]

#-------------------------------------------------------------------------------------

  # -- dataframe with just county storm events
  noaa_c = noaa_all[noaa_all['CZ_TYPE'] == 'C'][['STATE_CO_FIPS',dmgyr]]
  # -- renmae fips column
  noaa_c.columns = ['CO_FIPS',dmgyr]

  # -- stack county and zones dataframe to combine the data back together
  noaa_all2 = pd.concat([noaa_c,noaa_zm])

  # group by county FIPS with the zone->county FIPS and sum total damages per county for this year
  noaa_all2g = noaa_all2.groupby("CO_FIPS").sum().reset_index()
  noaa_all2g

  # merge onto compiled dataframe
  #noall = co_fips.reset_index().merge(noaa_all2g,left_on="FIPS", right_on="CO_FIPS", how ='outer').set_index('index')
  noall_shp = co_fips.merge(noaa_all2g,left_on="FIPS", right_on="CO_FIPS", how ='outer')
  co_shp[dmgyr] = noall_shp[dmgyr].copy() 

In [0]:
# co_shp.to_csv(r'/content/drive/My Drive/MLPP20 Project Data/Final Data/co_shp_noaa.csv', index = False)

In [0]:
# -- inflate damages amounts using cpi values
co_shp_uninf = co_shp.copy()
for i in range(1996, 2019):
  column = "dmg"+str(i)
  co_shp[column] = co_shp_uninf[column].apply(lambda x: cpi.inflate(x,i)).fillna(0).astype(int)

co_shp.head()

In [0]:
# -- sum up the total damage from 1996-2019
co_shp['dmg_sum'] = co_shp.iloc[:,10:].sum(axis=1)
co_shp.head()

In [0]:
co_shp.to_csv(r'/content/drive/My Drive/MLPP20 Project Data/Final Data/co_shp_inf.csv', index = False)

# GDP data

In [0]:
# -- *ATTENTION cell above outputs the results of this cell, which takes 40 MINUTES to run
# -- import library to pull data from xml
from bs4 import BeautifulSoup

# -- import county level gdp data(2001-2018) from BEA 
query = "https://apps.bea.gov/api/data/?UserID=2A52462F-92C3-4DFF-823A-0B5D5F9A5E75&method=GetData&datasetname=Regional&TableName=CAGDP2&LineCode=1&Year=ALL&GeoFips=COUNTY&ResultFormat=xml"

ct_gdp_request = requests.get(query)

ct_gdp_html = ct_gdp_request.content
ct_gdp_soup = BeautifulSoup(ct_gdp_html, 'xml')

# -- create data frame using BEA data(may take a while...)
GeoFips = []
TimePeriod = []
GDP = []

for i in range(len(ct_gdp_soup.Results.find_all('Data'))):
  GeoFips.append(ct_gdp_soup.Results.find_all('Data')[i]['GeoFips'])
  GDP.append(ct_gdp_soup.Results.find_all('Data')[i]['DataValue'])
  TimePeriod.append(ct_gdp_soup.Results.find_all('Data')[i]['TimePeriod'])

gdp_data = pd.DataFrame({'FIPS':GeoFips, 'Year':TimePeriod, 'GDP':GDP})

# -- pivot and format df
gdp_pivot = gdp_data[['Year', 'GDP']].pivot(columns='Year')['GDP']

gdpp01 = gdp_pivot.iloc[:,0].dropna().values
gdpp02 = gdp_pivot.iloc[:,1].dropna().values
gdpp03 = gdp_pivot.iloc[:,2].dropna().values
gdpp04 = gdp_pivot.iloc[:,3].dropna().values
gdpp05 = gdp_pivot.iloc[:,4].dropna().values
gdpp06 = gdp_pivot.iloc[:,5].dropna().values
gdpp07 = gdp_pivot.iloc[:,6].dropna().values
gdpp08 = gdp_pivot.iloc[:,7].dropna().values
gdpp09 = gdp_pivot.iloc[:,8].dropna().values
gdpp10 = gdp_pivot.iloc[:,9].dropna().values
gdpp11 = gdp_pivot.iloc[:,10].dropna().values
gdpp12 = gdp_pivot.iloc[:,11].dropna().values
gdpp13 = gdp_pivot.iloc[:,12].dropna().values
gdpp14 = gdp_pivot.iloc[:,13].dropna().values
gdpp15 = gdp_pivot.iloc[:,14].dropna().values
gdpp16 = gdp_pivot.iloc[:,15].dropna().values
gdpp17 = gdp_pivot.iloc[:,16].dropna().values
gdpp18 = gdp_pivot.iloc[:,17].dropna().values

gdp_df = pd.DataFrame({'FIPS': gdp_data['FIPS'].unique(), 'gdp2001':gdpp01, 'gdp2002':gdpp02, 'gdp2003':gdpp03, 'gdp2004':gdpp04, 'gdp2005':gdpp05, 'gdp2006':gdpp06, 'gdp2007':gdpp07, 'gdp2008':gdpp08, 'gdp2009':gdpp09, 'gdp2010':gdpp10, 'gdp2011':gdpp11, 'gdp2012':gdpp12, 'gdp2013':gdpp13, 'gdp2014':gdpp14, 'gdp2015':gdpp15, 'gdp2016':gdpp16, 'gdp2017':gdpp17, 'gdp2018':gdpp18})

# -- change df objects to int and multiply x1000 for true value
gdp_df['FIPS'] = gdp_df['FIPS'].astype(int)

gdp_cols = [col for col in gdp_df.columns if col not in ['FIPS']]

for col in gdp_cols:
  gdp_df[col] = pd.to_numeric(gdp_df[col].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
  gdp_df[col] = gdp_df[col] * 1000

gdp_df

In [0]:
gdp_df.to_csv(r'/content/drive/My Drive/MLPP20 Project Data/Final Data/bea_gdp.csv', index = False)

# CENSUS data

In [0]:
# -- set the needed variables and new column name
var_cs = ['B01003_001E','B02001_002E', 
          'B15003_017E','B15003_018E','B15003_019E','B15003_020E','B15003_021E','B15003_022E','B15003_023E','B15003_024E','B15003_025E',
          'B19013_001E','B25077_001E','B25001_001E',
          'B01001_003E','B01001_004E','B01001_005E','B01001_006E','B01001_007E','B01001_008E','B01001_009E','B01001_010E',
          'B01001_027E','B01001_028E','B01001_029E','B01001_030E','B01001_031E','B01001_032E','B01001_033E','B01001_034E']
var_cs_name = ['index','total_pop', 'white_pop', 
               'high_sch', 'ged_sch', 'col<1_sch', 'col1+_sch','ass_sch','bac_sch','mas_sch','prof_sch','doc_sch',
               'med_income', 'housing','total_houses',
               'm_age_<5','m_age_5-9','m_age_10-14','m_age_15-17','m_age_18-19','m_age_20','m_age_21','m_age_22-24',
               'f_age_<5','f_age_5-9','f_age_10-14','f_age_15-17','f_age_18-19','f_age_20','f_age_21','f_age_22-24',
               'year','county']

In [0]:
# -- get census data from 2012-2018
census_all = []
for i in range(2012, 2019):
  county_cs = cd.download('acs5', i, cd.censusgeo([('county', '*')]), var_cs).reset_index()
  county_cs['index'] = county_cs['index'].agg(str)
  county_cs['year'] = i


  # -- split index, get county and state index 
  sp1 =  county_cs['index'].str.split("> county:",expand=True)
  county_no = sp1[1]

  sp2 = sp1[0].str.split("state:",expand =True)
  state_no = sp2[1]
   
  # -- get countycode
  countycode_cs = (state_no+county_no).astype(int)
  county_cs['countycode']=countycode_cs 
  
  census_all.append(county_cs)

In [0]:
census_all = pd.concat(census_all, ignore_index=True)

In [0]:
census_all.columns = var_cs_name

In [0]:
census_all['total_pop_<25'] = census_all.iloc[:,15:31].sum(axis=1)
census_all['total_pop_>=25'] = census_all['total_pop'] - census_all['total_pop_<25']

census_all['total_ed_high+'] = census_all.iloc[:,3:12].sum(axis=1)

In [0]:
# -- romove housing values lower than 0
census_all = census_all.drop(census_all[census_all['housing']<0].index)
# -- note that in Y2012-2013 have 3221 counties, Y2014-2018 have 3220 counties.

In [0]:
# -- get the percentage of education level high than high school diplomas and white population
census_all['high_edu_pct'] = census_all['total_ed_high+']/census_all['total_pop_>=25'] *100
census_all['white_pct'] = census_all['white_pop']/census_all['total_pop'] *100
census_all['non-white_pct'] = 100 - census_all['white_pct']

In [0]:
census_all.to_csv(r'/content/drive/My Drive/MLPP20 Project Data/Final Data/census_all.csv', index = False)

# CENSUS annual data

In [0]:
# -- Import county population estimates from 2001-2018
popfile0010 = "http://www2.census.gov/programs-surveys/popest/datasets/2000-2010/intercensal/county/co-est00int-tot.csv?#"
popfile1019 = "https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv"

pop0010 = pd.read_csv(popfile0010, usecols = [i for i in range(19)], encoding = "ISO-8859-1")
pop1019 = pd.read_csv(popfile1019, usecols = [i for i in range(19)], encoding = "ISO-8859-1")

# -- Transform data to FIPS codes
pstate = pop0010["STATE"].astype(int).astype(str)
pcounty = pop0010["COUNTY"].astype(int).astype(str)
pcounty = "00"+ pcounty 
pcounty = pcounty.str[-3:]
pgeoid = (pstate + pcounty).astype(int)
pop0010["GEOID"] = pgeoid

p2state = pop1019["STATE"].astype(int).astype(str)
p2county = pop1019["COUNTY"].astype(int).astype(str)
p2county = "00"+ p2county 
p2county = p2county.str[-3:]
p2geoid = (p2state + p2county).astype(int)
pop1019["GEOID"] = p2geoid

pop0010 = pop0010.drop(["SUMLEV", "REGION", "DIVISION", "STATE", "COUNTY", "STNAME", "CTYNAME", "ESTIMATESBASE2000"], axis=1)
pop1019 = pop1019.drop(["SUMLEV", "REGION", "DIVISION", "STATE", "COUNTY", "STNAME", "CTYNAME", "CENSUS2010POP", "ESTIMATESBASE2010"], axis=1)

In [0]:
# -- merge with NOAA dataframe
pop_ts = pop0010.merge(pop1019,left_on="GEOID", right_on="GEOID")
co_shp_ts = co_shp.merge(pop_ts,left_on="GEOID", right_on="GEOID")

In [0]:
pop_ts.to_csv(r'/content/drive/My Drive/MLPP20 Project Data/Final Data/pop_ts.csv', index = False)
co_shp_ts.to_csv(r'/content/drive/My Drive/MLPP20 Project Data/Final Data/co_shp_ts.csv', index = False)

# FEMA NFIP data

In [0]:
# -- get the FEMA zipfile name
fname_fema = "https://www.fema.gov/media-library-data/1575491579309-2366ee38d902c1bc983370e132ee36cc/FIMA_NFIP_Redacted_Claims_Data_Set.zip"

zipfile = ZipFile(BytesIO(urlopen(fname_fema).read()))
zipfile.namelist()

In [0]:
# -- read the table
file = 'openFEMA_claims20190831.csv'
FEMA_claims_all = pd.read_csv(zipfile.open(file))

In [0]:
# -- extract useful columns and data 
FEMA_claims_sorted = FEMA_claims_all[FEMA_claims_all['yearofloss'] >= 1996]
FEMA_claims_sorted = FEMA_claims_sorted[['reportedcity','countycode', 'crsdiscount', 'yearofloss',
                                         'amountpaidonbuildingclaim','amountpaidoncontentsclaim','amountpaidonincreasedcostofcomplianceclaim',
                                         'totalbuildinginsurancecoverage', 'totalcontentsinsurancecoverage']]

In [0]:
# -- sum up and select useful columns
FEMA_claims_sorted['totalpaid']=FEMA_claims_sorted[['amountpaidonbuildingclaim','amountpaidoncontentsclaim','amountpaidonincreasedcostofcomplianceclaim']].sum(axis=1)
FEMA_claims_sorted['totalcovered']=FEMA_claims_sorted[['totalbuildinginsurancecoverage', 'totalcontentsinsurancecoverage']].sum(axis=1)
FEMA_claims = FEMA_claims_sorted[['reportedcity', 'countycode', 'crsdiscount', 'yearofloss','totalpaid', 'totalcovered']].copy()

In [0]:
FEMA_claims.to_csv(r'/content/drive/My Drive/MLPP20 Project Data/Final Data/FEMA_claims.csv', index = False)

In [0]:
# -- clean payout data to make the year in columns
NFIP_paid_year = FEMA_claims.groupby(["countycode","yearofloss"])['totalpaid'].sum().reset_index().pivot_table(values='totalpaid', index='countycode', columns='yearofloss').reset_index()

In [0]:
NFIP_paid_year.columns = NFIP_paid_year.columns.astype(str)

In [0]:
# adjust for inflation from 1996-2018
NFIP_paid_inf = NFIP_paid_year.copy()
for i in range(1996, 2019):
  column = str(i)
  NFIP_paid_inf[column] = NFIP_paid_year[column].apply(lambda x: cpi.inflate(x,i)).fillna(0).astype(int)

NFIP_paid_inf['paid_sum'] = NFIP_paid_inf.iloc[:,2:].sum(axis=1)

In [0]:
NFIP_paid_inf.to_csv(r'/content/drive/My Drive/MLPP20 Project Data/Final Data/NFIP_paid_inf.csv', index = False)

# NFIP policy count data

In [0]:
! wget https://www.fema.gov/media-library-data/1575509388770-d8a7d4c287aa6da25f742059141e8d47/FIMA_NFIP_Redacted_Policies_Data_Set_Part_1.zip
! wget https://www.fema.gov/media-library-data/1575509870335-4b7bcf6544a6dea594a255e01234a23a/FIMA_NFIP_Redacted_Policies_Data_Set_Part_2.zip
! wget https://www.fema.gov/media-library-data/1575510711995-d524a8a137526489920556dd3453fd13/FIMA_NFIP_Redacted_Policies_Data_Set_Part_3.zip
! wget https://www.fema.gov/media-library-data/1575509883146-4b7bcf6544a6dea594a255e01234a23a/FIMA_NFIP_Redacted_Policies_Data_Set_Part_4.zip
! wget https://www.fema.gov/media-library-data/1575510718199-d524a8a137526489920556dd3453fd13/FIMA_NFIP_Redacted_Policies_Data_Set_Part_5.zip
! wget https://www.fema.gov/media-library-data/1575509900847-4b7bcf6544a6dea594a255e01234a23a/FIMA_NFIP_Redacted_Policies_Data_Set_Part_6.zip

In [0]:
!unzip \*.zip

In [0]:
import pandas as pd

'1'.zfill(2)

In [0]:
! cp /content/openFEMA_policies20190831_01.csv '/content/drive/My Drive/Dongyang/Final_project/'

In [0]:
! cp /content/openFEMA_policies20190831_02.csv '/content/drive/My Drive/Dongyang/Final_project/'

In [0]:
! cp /content/openFEMA_policies20190831_03.csv '/content/drive/My Drive/Dongyang/Final_project/'

In [0]:
! cp /content/openFEMA_policies20190831_04.csv '/content/drive/My Drive/Dongyang/Final_project/'

In [0]:
! cp /content/openFEMA_policies20190831_05.csv '/content/drive/My Drive/Dongyang/Final_project/'

In [0]:
! cp /content/openFEMA_policies20190831_06.csv '/content/drive/My Drive/Dongyang/Final_project/'

In [0]:
! cp /content/openFEMA_policies20190831_07.csv '/content/drive/My Drive/Dongyang/Final_project/'

In [0]:
! cp /content/openFEMA_policies20190831_08.csv '/content/drive/My Drive/Dongyang/Final_project/'

In [0]:
! cp /content/openFEMA_policies20190831_09.csv '/content/drive/My Drive/Dongyang/Final_project/'

In [0]:
! cp /content/openFEMA_policies20190831_10.csv '/content/drive/My Drive/Dongyang/Final_project/'

In [0]:
! cp /content/openFEMA_policies20190831_11.csv '/content/drive/My Drive/Dongyang/Final_project/'

In [0]:
import pandas as pd

df_ls = []
for i in range(1, 4):
  file_path = '/content/drive/My Drive/Dongyang/Final_project/openFEMA_policies20190831_' + str(i).zfill(2) + '.csv'
  df = pd.read_csv(file_path)
  df_ls.append(df)

df = pd.concat(df_ls)

In [0]:
df_ls = []
df.shape

9 dfs have been loaded

In [0]:
df.head()

In [0]:
name_ls = df.columns.to_list()
print(df.columns.to_list())

In [0]:
col_to_keep = ['countycode', 'policyeffectivedate', 'policyterminationdate']

In [0]:
name_ls.index('countycode')

In [0]:
name_ls.index('policyeffectivedate')

In [0]:
name_ls.index('policyterminationdate')

In [0]:
df.iloc[:,[7, 31, 32]]

In [0]:
import pandas as pd
file_path = '/content/drive/My Drive/Dongyang/Final_project/openFEMA_policies20190831_01.csv'
df1 = pd.read_csv(file_path)

In [0]:
col_to_keep = [7, 9, 10, 30, 31, 32]

In [0]:
df1.columns.to_list()

In [0]:
df1.columns.to_list().index('deductibleamountincontentscoverage')

In [0]:
df1.columns.to_list().index('deductibleamountinbuildingcoverage')

In [0]:
df1_f = df1.iloc[:, col_to_keep].copy()
df1_f.info()
df1.info()

In [0]:
file_path = '/content/drive/My Drive/Dongyang/Final_project/openFEMA_policies20190831_02.csv'
df2 = pd.read_csv(file_path)

In [0]:
df2.info()

In [0]:
df2_f = df2.iloc[:, col_to_keep].copy()
df2_f.columns = df1_f.columns
df2_f.head()

In [0]:
col_names = df1_f.columns.to_list()
col_names

In [0]:
df_ls = []
for i in range(1, 12):
  file_path = '/content/drive/My Drive/Dongyang/Final_project/openFEMA_policies20190831_' + str(i).zfill(2) + '.csv'
  df = pd.read_csv(file_path)
  df = df.iloc[:, col_to_keep]
  df.columns = col_names
  df_ls.append(df)

df = pd.concat(df_ls)

In [0]:
df.head()

In [0]:
df.shape

In [0]:
df.info()

In [0]:
deductibleamountinbuildingcoverage_dict = {
    '0': 500,
    '1': 1000,
    '2': 2000,
    '3': 3000,
    '4': 4000,
    '5': 5000,
    '9': 750,
    'A': 10000,
    'B': 15000,
    'C': 20000,
    'D': 25000,
    'E': 50000,
    'F': 1250,
    'G': 1500
}

In [0]:
df['deductibleamountinbuildingcoverage__'] = df['deductibleamountinbuildingcoverage'].astype(str).apply(lambda x: deductibleamountinbuildingcoverage_dict[x])

In [0]:
df_2 = df.copy().drop(columns=['deductibleamountinbuildingcoverage',
                               'deductibleamountincontentscoverage'])
df_2.dropna(inplace=True)

In [0]:
df_2.isnull().sum()

In [0]:
df_2['policyeffectivedate'] = pd.to_datetime(df_2['policyeffectivedate'])
df_2['policyterminationdate'] = pd.to_datetime(df_2['policyterminationdate'])
df_2.info()

In [0]:
df_2['policyeffectivedate_yr'] = df_2['policyeffectivedate'].dt.year
df_2['policyterminationdate_yr'] = df_2['policyterminationdate'].dt.year
df_2.head()

In [0]:
df_2['delta_yr'] = df_2['policyterminationdate_yr'] - df_2['policyeffectivedate_yr']
df_2.head()

In [0]:
import seaborn as sns
sns.distplot(df_2['delta_yr'])

In [0]:
df_2.sort_values('policyeffectivedate_yr')

In [0]:
df_2.to_csv('df_extracted.csv')

In [0]:
df_2.reset_index(drop=True)
df_2

In [0]:
def yr_ls(x):
  return list(np.arange(x[0], x[1]+1))

df_2['yr_range'] = df_2[['policyeffectivedate_yr', 'policyterminationdate_yr']].apply(yr_ls, axis=1)
df_2.head()

In [0]:
import numpy as np
list(np.arange(1, 10))

In [0]:
import pandas as pd
df_2 = pd.read_csv('/content/df_extracted.csv')

In [0]:
df_2= df_2[df_2['policyterminationdate_yr']>=2012]

In [0]:
# df_2_test = df_2[(df_2['policyeffectivedate_yr']>=2012) & (df_2['policyterminationdate_yr']<2013)]
df_2.drop(columns=['policyeffectivedate', 'policyterminationdate',
                   'policyterminationdate_yr', 'Unnamed: 0'], inplace=True)
df_2.head()

In [0]:
df_2.rename(columns={'policyeffectivedate_yr': 'year1'}, inplace=True)
df_2.head()

In [0]:
df_2['year2'] = df_2['year1'] + 1
df_2['year3'] = df_2['year1'] + 2
df_2['year4'] = df_2['year1'] + 3
df_2.head()

In [0]:
change_col_name = {
    'year1': 'year',
    'year2': 'year',
    'year3': 'year',
    'year4': 'year',
}

In [0]:
year1_df = df_2[['countycode', 'policycount', 'year1']].copy()
year1_df.rename(columns=change_col_name, inplace=True)
year1_df = year1_df.groupby(['countycode', 'year']).sum().reset_index()
year1_df.head()

In [0]:
year2_df = df_2[['countycode', 'policycount', 'year2', 'delta_yr']].copy()
year2_df = year2_df[year2_df['delta_yr'] >= 1].drop(columns='delta_yr')
year2_df.rename(columns=change_col_name, inplace=True)
year2_df = year2_df.groupby(['countycode', 'year']).sum().reset_index()
year2_df.head()

In [0]:
year3_df = df_2[['countycode', 'policycount', 'year3', 'delta_yr']].copy()
year3_df = year3_df[year3_df['delta_yr'] >= 2].drop(columns='delta_yr')
year3_df.rename(columns=change_col_name, inplace=True)
year3_df = year3_df.groupby(['countycode', 'year']).sum().reset_index()
year3_df.head()

In [0]:
year4_df = df_2[['countycode', 'policycount', 'year4', 'delta_yr']].copy()
year4_df = year4_df[year4_df['delta_yr'] >= 3].drop(columns='delta_yr')
year4_df.rename(columns=change_col_name, inplace=True)
year4_df = year4_df.groupby(['countycode', 'year']).sum().reset_index()
year4_df.head()

In [0]:
year_ls = [year1_df, year2_df, year3_df, year4_df]
combine_df = pd.concat(year_ls)
combine_df.head()

In [0]:
combine_df.shape

In [0]:
combine_df.groupby(['countycode', 'year']).sum()

combine_df.head()

In [0]:
combine_df.to_csv('policy_count.csv')